<a href="https://colab.research.google.com/github/DipankarJDutta/DipankarJDutta-Biomarker-discovery-from-B-T-cells-in-FASD-GDM-with-LSTM/blob/Feature-extraction-with-Shap/LSTM_B_T_Median_optimized%20model_SHAP%20feature%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Fixing random seed to 007 for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [98]:
#Installing SHAP (SHapley Additive exPlanations)
!pip install shap

In [0]:
#Import SHAP specific essentials
from __future__ import print_function
from keras.preprocessing import sequence
from keras.layers import Embedding


In [0]:
# Loading essentials
from numpy import loadtxt
from numpy import reshape
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:
# Load training data
x_train = loadtxt('B-T-median.csv', delimiter = ',')
x_test = loadtxt('B-T-median.csv', delimiter = ',')

In [102]:
# Reshape 2D training dataset into a 3D dataset with columns as features with one time-step
x_train = x_train.reshape(56, 1, 30)
x_test = x_test.reshape(56, 1, 30)

# Print dataset shape
x_train.shape
x_test.shape

(56, 1, 30)

In [0]:
#Specify binary output of good (1) and bad (0) learners, y_train and y_test, in x_train and x_test respectively. Learner type differentiated by population median.
y_train = x_train [:, :, -1]
y_test = x_test [:, :, -1]


In [0]:
# Define LSTM One-to-One Model with 3 LSTM cells (determined via hyperparameter optimization). Sigmoid activation on the output layer for binary classification.
model = Sequential()
model.add(LSTM(3, input_shape=(1,30)))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [106]:
#Fit the model with all of the data as train and test samples
model.fit(x_train, y_train, batch_size = 8, epochs = 1000, shuffle = True, validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=8)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 56 samples, validate on 56 samples
Epoch 1/1000
56/56 [==============================] - 5s 87ms/step - loss: 0.7107 - acc: 0.4464 - val_loss: 0.7086 - val_acc: 0.4464
Epoch 2/1000
56/56 [==============================] - 0s 732us/step - loss: 0.7075 - acc: 0.4464 - val_loss: 0.7059 - val_acc: 0.4464
Epoch 3/1000
56/56 [==============================] - 0s 586us/step - loss: 0.7051 - acc: 0.4464 - val_loss: 0.7033 - val_acc: 0.4464
Epoch 4/1000
56/56 [==============================] - 0s 601us/step - loss: 0.7023 - acc: 0.4464 - val_loss: 0.7012 - val_acc: 0.4464
Epoch 5/1000
56/56 [==============================] - 0s 592us/step - loss: 0.7008 - acc: 0.4464 - val_loss: 0.6989 - val_acc: 0.4464
Epoch 6/1000
56/56 [==============================] - 0s 667us/step - loss: 0.6982 - acc: 0.4464 - val_loss: 0.6972 - val_acc: 0.4464
Epoch 7/1000
56/56 [==============================] - 0s 649us/step - loss: 0.6966 - acc: 0.4464 - val_loss: 0.6954 - val_acc: 0.4464
Epoch 8/1000
56/56 

In [113]:
# Explain model with DeepExplainer from SHAP
#Get Shap values
import shap
explainer = shap.DeepExplainer(model, x_train)
shap_values = explainer.shap_values(x_test)
type(shap_values)

list

In [115]:
# Display shap values in the list
print(*shap_values, sep = ",")

[[[-2.51219557e-03 -2.27496986e-03 -1.48658481e-04  1.27347506e-03
   -8.50019385e-05  7.95999006e-04  3.97042365e-02  1.01045268e-03
   -6.06285510e-04  2.39167915e-03  2.29205514e-03 -2.98179297e-03
   -1.00642034e-03 -2.08973510e-03  6.43792900e-04  2.16353625e-02
   -2.48161461e-05 -2.54772286e-05  2.77993084e-03 -3.21642550e-03
   -2.45635091e-03  2.76583089e-02 -1.98449379e-04  4.77258409e-06
    4.42498552e-04  2.16146497e-04 -2.32330365e-03 -3.34547065e-03
   -5.05778758e-03  3.87360877e-01]]

 [[-3.19704962e-04 -4.43056188e-04 -1.64314483e-04  1.02896393e-03
    6.56010653e-05  3.62082918e-04  4.32706280e-02  9.25359295e-04
    4.35251580e-04  3.69192591e-03  2.03134359e-03 -1.25149946e-02
    1.03858191e-03 -1.82946714e-03 -5.89235180e-05  1.08674489e-02
   -1.08794936e-03  1.35197152e-03  5.38783990e-03  1.06210932e-02
    2.15070802e-02 -2.78651206e-02  2.33708571e-03 -7.17548532e-04
   -1.31677400e-04 -9.41905399e-03  7.63810194e-03 -5.57754276e-03
    2.15265084e-02  3.85